## Verifying the irreducible representations of alternating group A4

There are 4 conjugacy classes in A4, and 4 irreducible representations of degrees 1, 1, 1, and 3, from Problem 2.12.17 in Sagan.

In [1]:
# Initialization

import numpy as np
import numpy.linalg as npla

A4 = AlternatingGroup(4)
print(f"A4: {A4}")
print(f" {A4[:]}")
print()

# Get a primitive cube root of unity. 
# Probably should do this symbolically somehow in Sage, but for now just use floats.

q = (-1 + 1j*np.sqrt(3)) / 2
qq = q*q

print("Cube roots of unity:")
print(f" q:   {q}")
print(f" q^2: {qq}")
print(f" q^3:  {qq*q}")
print()

# A crude test for close-enough-to-0 in well-conditioned floating-point

def is_tiny(x):
    return npla.norm(x) < 10**-8

print("is_tiny( q^3 - 1 ):", is_tiny(qq*q-1))
print()

A4: Alternating group of order 4!/2 as a permutation group
 [(), (1,2)(3,4), (1,3)(2,4), (1,4)(2,3), (2,3,4), (1,2,4), (1,3,2), (1,4,3), (2,4,3), (1,2,3), (1,3,4), (1,4,2)]

Cube roots of unity:
 q:   -0.500000000000000 + 0.866025403784439*I
 q^2: -0.500000000000000 - 0.866025403784439*I
 q^3:  1.00000000000000 + 1.11022302462516E-16*I

is_tiny( q^3 - 1 ): True



In [2]:
# Verify that the 1-D representations (which are also characters) from Problem 2.12.17 all work.

chis = [[1, 1, 1, 1], [1, 1, q, qq], [1, 1, qq, q]]

for chi in chis:
    
    print('Character', chi)
    rep = {}
    for i, c in enumerate(A4.conjugacy_classes()):
        print('  ', c[:], ':', chi[i])
        for g in c:
            rep[g] = chi[i]
    
    for g in A4:
        for h in A4:
            assert(is_tiny(rep[g]*rep[h] - rep[g*h]))
            
    print('Multiplication ok for this character!')
    print()

Character [1, 1, 1, 1]
   [()] : 1
   [(1,2)(3,4), (1,3)(2,4), (1,4)(2,3)] : 1
   [(1,2,3), (1,4,2), (1,3,4), (2,4,3)] : 1
   [(1,2,4), (1,4,3), (2,3,4), (1,3,2)] : 1
Multiplication ok for this character!

Character [1, 1, -0.500000000000000 + 0.866025403784439*I, -0.500000000000000 - 0.866025403784439*I]
   [()] : 1
   [(1,2)(3,4), (1,3)(2,4), (1,4)(2,3)] : 1
   [(1,2,3), (1,4,2), (1,3,4), (2,4,3)] : -0.500000000000000 + 0.866025403784439*I
   [(1,2,4), (1,4,3), (2,3,4), (1,3,2)] : -0.500000000000000 - 0.866025403784439*I
Multiplication ok for this character!

Character [1, 1, -0.500000000000000 - 0.866025403784439*I, -0.500000000000000 + 0.866025403784439*I]
   [()] : 1
   [(1,2)(3,4), (1,3)(2,4), (1,4)(2,3)] : 1
   [(1,2,3), (1,4,2), (1,3,4), (2,4,3)] : -0.500000000000000 - 0.866025403784439*I
   [(1,2,4), (1,4,3), (2,3,4), (1,3,2)] : -0.500000000000000 + 0.866025403784439*I
Multiplication ok for this character!



In [4]:
# Verify the 3-D representation works.

# The 3-D rep is the orthogonal complement of the trivial rep in the defining rep.
# The corresponding character is the number of fixed points - 1, i.e. [3, -1, 0, 0]

# Matrix to change basis from (1,2,3,4) to (1+2+3+4, 2-1, 3-1, 4-1)
T = np.array([[1,1,1,1],[-1,1,0,0],[-1,0,1,0],[-1,0,0,1]])
print('T:\n',T)

# ... and its inverse
Ti = npla.inv(T)
print()
print('Ti:\n',Ti)
print()

rep = {}
unit = np.array([1,0,0,0])
for g in A4:
    
    print('g:', g)
    M = g.matrix()
    print('defining rep:')
    print(M)
    
    # conjugate M by T to change the basis
    TMTi = T@M@Ti
    
    # check that we really got the block decomposition
    assert is_tiny(TMTi[0,:] - unit)  
    assert is_tiny(TMTi[:,0] - unit)
    
    # the 3D representation is the 3-by-3 block
    rep[g] = TMTi[1:,1:]
    print('3D rep:')
    print(rep[g])
    print('chi(g) =', np.trace(rep[g]))
    print()

print('Decomposition ok!\n')

# check the whole multiplication table
for g in A4:
    for h in A4:
        assert(is_tiny(rep[g]@rep[h] - rep[g*h]))
        
print('Multiplication ok!\n')

T:
 [[ 1  1  1  1]
 [-1  1  0  0]
 [-1  0  1  0]
 [-1  0  0  1]]

Ti:
 [[ 0.25 -0.25 -0.25 -0.25]
 [ 0.25  0.75 -0.25 -0.25]
 [ 0.25 -0.25  0.75 -0.25]
 [ 0.25 -0.25 -0.25  0.75]]

g: ()
defining rep:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]
3D rep:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
chi(g) = 3.0

g: (1,2)(3,4)
defining rep:
[0 1 0 0]
[1 0 0 0]
[0 0 0 1]
[0 0 1 0]
3D rep:
[[-1.  0.  0.]
 [-1.  0.  1.]
 [-1.  1.  0.]]
chi(g) = -1.0

g: (1,3)(2,4)
defining rep:
[0 0 1 0]
[0 0 0 1]
[1 0 0 0]
[0 1 0 0]
3D rep:
[[ 0. -1.  1.]
 [ 0. -1.  0.]
 [ 1. -1.  0.]]
chi(g) = -1.0

g: (1,4)(2,3)
defining rep:
[0 0 0 1]
[0 0 1 0]
[0 1 0 0]
[1 0 0 0]
3D rep:
[[ 0.  1. -1.]
 [ 1.  0. -1.]
 [ 0.  0. -1.]]
chi(g) = -1.0

g: (2,3,4)
defining rep:
[1 0 0 0]
[0 0 1 0]
[0 0 0 1]
[0 1 0 0]
3D rep:
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
chi(g) = 0.0

g: (1,2,4)
defining rep:
[0 1 0 0]
[0 0 0 1]
[0 0 1 0]
[1 0 0 0]
3D rep:
[[-1.  0.  1.]
 [-1.  1.  0.]
 [-1.  0.  0.]]
chi(g) = 0.0

g: (1,3,2)
defining rep:
[0 0 1 